In [5]:
import json
import os
import os.path as osp
import time
from typing import List, Dict, Union
from llm import get_response_from_llm, extract_json_between_markers

import requests
import backoff

from prompt import idea_first_prompt, idea_reflection_prompt

In [6]:
S2_API_KEY = os.getenv("S2_API_KEY")